<a href="https://colab.research.google.com/github/wasp33/ximen0703/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from urllib.request import urlretrieve
url="https://github.com/Elwing-Chou/ximen0703/raw/main/titanic/test.csv"
urlretrieve(url,"test.csv")
url="https://github.com/Elwing-Chou/ximen0703/raw/main/titanic/train.csv"
urlretrieve(url,"train.csv")

('train.csv', <http.client.HTTPMessage at 0x7ff86df85c00>)

處理前先分類
1. 類別型 Pclass,Name(MID),Sex,Cabin,Embarked
2. 數值型 Age,Sibsp,Parch,ticket(count), Fare
類別最可能: 最常出現的值
數值最可能:

偏度定義; $ \frac{1}{n} \sum_{i=1}^n (P_i-avg)^3 $

GDP => 平均

薪資 => 中位數

成績 => 中位數

In [43]:
import pandas as pd
datas = pd.read_csv("train.csv", encoding="utf-8")
data_predict = pd.read_csv("test.csv",encoding="utf-8")
#data = pd.DataFrame(datas)

In [44]:
data_total=pd.concat([datas,data_predict],axis=0)
#data_total

data_total=data_total.drop(["PassengerId","Survived"], axis=1)
data_total

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


isna() 檢查是否為空值
isna().sum() 總數量
[]代入[]陣列

In [45]:
# 一維: Series
s=data_total.isna().sum()
f = s>0
s[f].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [46]:
d = pd.DataFrame([
    [1,2],
    [3,4],
    [5,6]])
d[[True,False,True]]

,0,1
0,1,2
2,5,6


In [47]:
#value_counts()
most=data_total["Embarked"].value_counts().idxmax()
data_total["Embarked"] = data_total["Embarked"].fillna(most)

In [48]:
# 語法demo:
# list型態 [0]操作
# function型態 (4.2)操作
b = int
b(4.2)

4

In [49]:
# pandas demo:
# Series.apply(func型態)
def square(n):
    return n * n
s = pd.Series([1, 2, 3, 4])
s.apply(square)

0     1
1     4
2     9
3    16
dtype: int64

In [50]:
#
def name(n):
    return n.split(",")[-1].split(".")[0].strip()
data_total["Name"]=data_total["Name"].apply(name)
#data_total["Name"]

In [51]:
counts = data_total["Ticket"].value_counts()
def ticket(t):
    return counts[t]
data_total["Ticket"] = data_total["Ticket"].apply(ticket)
#data_total

In [52]:
def cabin(c):
    if pd.isna(c):
        return c
    else:
        return c[0]
data_total["Cabin"] = data_total["Cabin"].apply(cabin)
#data_total

In [53]:
#補數值中位數
med = data_total.median().drop("Pclass")
data_total = data_total.fillna(med)
#再檢查一次空值有哪些
s=data_total.isna().sum()
f = s>0
s[f].sort_values(ascending=False)

<ipython-input-53-7ccf62f78ea1>:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  med = data_total.median().drop("Pclass")


Cabin    1014
dtype: int64

In [61]:
counts = data_total["Name"].value_counts()
reserved = counts[counts > 50].index
def name2(mid):
    if mid in reserved:
        return mid
    else:
        return None
data_total["Name"]=data_total['Name'].apply(name2)
#data_total["Name"]

0          Mr
1         Mrs
2        Miss
3         Mrs
4          Mr
        ...  
413        Mr
414      None
415        Mr
416        Mr
417    Master
Name: Name, Length: 1309, dtype: object

In [62]:
pd.set_option('display.max_columns',500)
data_total = pd.get_dummies(data_total)
data_total = pd.get_dummies(data_total,columns=["Pclass"])
data_total

In [66]:
#family = SibSp + Parch
data_total['Family'] = data_total["SibSp"] + data_total["Parch"]
data_total['Family'].value_counts()

0     790
1     235
2     159
3      43
5      25
4      22
6      16
10     11
7       8
Name: Family, dtype: int64

In [54]:
# demo concat
d1 = pd.DataFrame([[1, 2]])
d2 = pd.DataFrame([[3, 4]])
pd.concat([d1, d2], axis=0)

,0,1
0,1,2
0,3,4


In [ ]:
# demo: loc/iloc

In [ ]:
x = data_total.iloc[:len(datas)]
y = datas["Survived"]
x_predict = data_total.iloc[len(datas)]

In [55]:
#from sklearn.model_selection import train_test_split
#x = datas.drop("Survived", axis=1)
#y = datas["Survived"]
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)